In [ ]:
!git clone https://github.com/AlexBorger/block-zone-simulator.git

In [ ]:
import sys, os
module_path = os.path.abspath('block-zone-simulator/Code/') 
if module_path not in sys.path:
    sys.path.append(module_path)

from circuit import Circuit
import numpy as np
import pandas as pd

blocks = {
    'Load': {
        'next_block': 'A Lift',
        'seconds_to_reach_block': 8,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': None,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': True,
        'hold_time': 34,  # true is ~ 34?
        'has_merger_switch': False,
        'has_splitter_switch': False
    },
    'A Lift': {
        'next_block': 'B Lift',
        'seconds_to_reach_block': 14,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': 4,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None,
        'has_merger_switch': False,
        'has_splitter_switch': False
    },
    'B Lift': {
        'next_block': 'Zone 1',
        'seconds_to_reach_block': 36,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': 4,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None,
        'has_merger_switch': False,
        'has_splitter_switch': False
    },
    'Zone 1': {
        'next_block': 'Zone 2',
        'seconds_to_reach_block': 11,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': None,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': True,
        'hold_time': 9,
        'has_merger_switch': False,
        'has_splitter_switch': False
    },
    'Zone 2': {
        'next_block': 'Zone 3',
        'seconds_to_reach_block': 18,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': None,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': True,
        'hold_time': 10,
        'has_merger_switch': False,
        'has_splitter_switch': False
    },
    'Zone 3': {
        'next_block': 'Hold',
        'seconds_to_reach_block': 24,
        'seconds_to_clear_from_held': 7,
        'seconds_to_clear_block_in_motion': 2,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None,
        'has_merger_switch': False,
        'has_splitter_switch': False
    },
    'Hold': {
        'next_block': 'Unload',
        'seconds_to_reach_block': 24,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': 4,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': False,
        'hold_time': None,
        'has_merger_switch': False,
        'has_splitter_switch': False
    },
    'Unload': {
        'next_block': 'Load',
        'seconds_to_reach_block': 6,
        'seconds_to_clear_from_held': 6,
        'seconds_to_clear_block_in_motion': None,
        'is_occupied': False,
        'can_operate_from_stop': True,
        'mandatory_hold': True,
        'hold_time': 10,  # was 20
        'has_merger_switch': False,
        'has_splitter_switch': False
    }
}

optional_params = {
    'sluggishness': True,
    'sluggishness_mu': 1.5,
    'sluggishness_sigma': 0.6,
    'random_seed': 10,
    'circuit_completion_blocks': ['Load']
}

def calculate_hourly_capacity(vehicle_capacity, circuit):
    avg_cycles_completed = sum([circuit.trains[train].circuits_completed for train in circuit.trains]) / len(circuit.trains)
    avg_cycles_per_hour = avg_cycles_completed * 3600 / circuit.time
    total_cycles_per_hour = avg_cycles_per_hour * len(circuit.trains)
    total_hourly_capacity = vehicle_capacity * total_cycles_per_hour
    return total_hourly_capacity

theoretical_capacities = []
theoretical_idle_percents = []
actual_capacities = []
actual_idle_percents = []

sim_run_time = 72000  # 10 hours (60s * 60min * 10hr)
vehicle_capacity = 34

for n in [1, 2, 3, 4, 5]:
    # run without sluggishness
    optional_params['sluggishness'] = False
    circuit = Circuit(blocks, num_trains=n, optional_params=optional_params)
    for _ in range(sim_run_time):
        circuit.step()
    # analyze results
    theoretical_capacities.append(calculate_hourly_capacity(vehicle_capacity=vehicle_capacity, circuit=circuit))
    train_idles = []
    for train_name in circuit.trains:
        train_idles.append(100*circuit.trains[train_name].total_seconds_held / circuit.time)
    theoretical_idle_percents.append(round(np.mean(train_idles),2))
    # run again with sluggishness
    optional_params['sluggishness'] = True
    circuit = Circuit(blocks, num_trains=n, optional_params=optional_params)
    for _ in range(sim_run_time):
        circuit.step()
    actual_capacities.append(calculate_hourly_capacity(vehicle_capacity=vehicle_capacity, circuit=circuit))
    train_idles = []
    for train_name in circuit.trains:
        train_idles.append(100*circuit.trains[train_name].total_seconds_held / circuit.time)
    actual_idle_percents.append(round(np.mean(train_idles),2))

df = pd.DataFrame({
    'num_trains': list(range(1,6)),
    'theoretical_capacity': theoretical_capacities,
    'actual_capacity': actual_capacities,
    'actual_to_theoretical_ratio': [actual_capacities[i] / theoretical_capacities[i] for i in range(len(actual_capacities))],
    'theoretical_idle_percents': theoretical_idle_percents,
    'actual_idle_percents': actual_idle_percents
})

df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output

# Sim Data
num_trains = 4
num_timesteps = 360
train_blocks = None  # Store current block
optional_params = {
    'random_seed': 10,
    'circuit_completion_blocks': ['Load']
}
circuit = Circuit(blocks, num_trains=num_trains, optional_params=optional_params)

# Initialize the plot
plt.figure(figsize=(10, 6))
plt.xlabel("Timestep")
plt.ylabel("Block")
plt.title("Train Movements on Track (Real-time)")
plt.ylim(0, 4)  # Adjust y-limit based on your block range
plt.xlim(0, 5)
plt.ion()  # Turn on interactive mode

block_locs = {
    'Load': (1, 1),
    'A Lift': (1, 2),
    'B Lift': (2, 2),
    'Zone 1': (3, 2),
    'Zone 2': (4, 2),
    'Zone 3': (4, 1),
    'Hold': (3, 1),
    'Unload': (2, 1)
}

block_txt = {block: block for block in circuit.blocks}

train_color_list = [ 'green', 'blue', 'orange', 'purple', 'black']  # List of colors
train_colors = dict()
i = 0
for t in circuit.trains:
    train_colors[t] = train_color_list[i]
    i += 1

# Simulation loop
for t in range(num_timesteps):
    # Simulate train movement (replace with your actual logic)
    circuit.step()
    train_blocks = [block_locs[circuit.trains[t].current_block] for t in circuit.trains]
    
    # Clear the previous plot
    clear_output(wait=True)
    
    # show labels for block areas
    for b in block_locs:
        block_x, block_y = block_locs[b]
        plt.text(block_x, block_y - 0.2, block_txt[b], ha='center', va='top')
    plt.text(2.5, 3.6, f'Time: {t} seconds', ha='center', va='top')
    
    # Update the plot
    held_trains = [t for t in circuit.trains if circuit.trains[t].current_status == 'held' and circuit.trains[t].seconds_held_at_current_block > 0]
    not_held_trains = [t for t in circuit.trains if t not in held_trains]
    not_held_trains_blocks = [block_locs[circuit.trains[t].current_block] for t in not_held_trains]
    held_trains_blocks = [block_locs[circuit.trains[t].current_block] for t in held_trains]
    plt.scatter([x[0] for x in not_held_trains_blocks], [x[1] for x in not_held_trains_blocks], 
                c=[train_colors[t] for t in circuit.trains if t in not_held_trains], marker='s', s=150)
    plt.scatter([x[0] for x in held_trains_blocks], [x[1] for x in held_trains_blocks], 
                c=[train_colors[t] for t in circuit.trains if t in held_trains], marker='s', s=150, 
                edgecolors='red', # Color of the square boundaries
                linewidths=1.5)
    plt.ylim(0, 4)  # Adjust y-limit based on your block range
    plt.xlim(0, 5)
    
    plt.draw()
    plt.pause(0.01)  # Adjust pause time for animation speed

plt.ioff()  # Turn off interactive mode
plt.show()